# Use Case 5: Gene Set Enrichment Analysis

## Step 1: Importing packages and setting up your notebook.

We start the notebook by importing the standard packages for data science. These are useful for analyzing data stored in dataframes and for plotting the results.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import gseapy as gp
from gseapy.plot import barplot, dotplot

Our cancer data and a variety of accessory functions are provided in the cptac package.

In [2]:
import cptac
en = cptac.Ucec()

## Step 2: Joining dataframes

For this example we will be separating the protein abudance based on whether the sample is tumor or non-tumor tissue. This information is stored in the "discovery_study/type_of_analyzed_samples" column in the clinical dataframe. We first join the clinical information into the proteomics dataframe utilizing the `en.join_metadata_to_omics` function.

In [3]:
tumorProt = en.join_metadata_to_omics(metadata_name="clinical",
                                      metadata_source='mssm',
                                      metadata_cols='discovery_study/type_of_analyzed_samples',
                                      omics_name="proteomics",
                                      omics_source='umich')

## Step 3: Organizing the data

We then separate the proteomics into two groups, based on whether the patient is categorized as "Tumor" or "Tumor_and_Normal". We make a single value out of this in the code below calling everything besides "Tumor" a "Normal" sample.

In [4]:
#Retrieve boolean array of true values
tumor_bool = tumorProt['discovery_study/type_of_analyzed_samples'] == "Tumor"
normal_bool = tumorProt['discovery_study/type_of_analyzed_samples'] != "Tumor"
#Use boolean array to select for appropriate patients
tumor = tumorProt[tumor_bool]
normal = tumorProt[normal_bool]

## Step 4: Perform statistical tests

Next, we find the genes that are upregulated in each partition using Welch's t-test (a variation on the two sample t-test due to different variances between the two groups) to compare the "Tumor" individuals with the "Normal" individuals for each gene.

In [ ]:
#Create array variables to hold the significant genes for each partition
tumor_genes = []
normal_genes = []
#Grab the genes of interest, ignoring the MSI column in the dataframe
genes = tumor.columns[1:]
#Correct alpha level for multiple testing by dividing the standard .05 by the number of genes to be analyzed
threshold = .05 / len(genes)
#Perform Welch's t-test(different variances) on each gene between the two groups
for gene in genes:
    tumor_gene_abundance = tumor[gene]
    normal_gene_abundance = normal[gene]
    
    if len(tumor_gene_abundance.shape) > 1 or len(normal_gene_abundance.shape) > 1:
        # take the mean across the columns
        tumor_gene_abundance = tumor_gene_abundance.mean(axis=1)
        normal_gene_abundance = normal_gene_abundance.mean(axis=1)
    
    pvalue = stats.ttest_ind(tumor_gene_abundance, normal_gene_abundance, equal_var=False, nan_policy='omit').pvalue
    #If the P-value is significant, determine which partition is more highly expressed
    if pvalue < threshold:
        if tumor_gene_abundance.mean() > normal_gene_abundance.mean():
            tumor_genes.append(gene.split("_")[0])
        elif normal_gene_abundance.mean() > tumor_gene_abundance.mean():
            normal_genes.append(gene.split("_")[0])
#Optional check of number of genes in each partition
print("Proteomics Tumor Genes:", len(tumor_genes))
print("Proteomics Normal Genes:", len(normal_genes))

## Step 5: Gene set enrichment analysis

We then use the genes that are up-regulated in these partitions to perform a Gene Set Enrichment Analysis using the `gp.enrichr()` function (`gp` being the specified abbreviation for the imported gseapy package).

In [ ]:
tumor_enr = gp.enrichr(gene_list=tumor_genes, gene_sets='KEGG_2016', outdir='test/enrichr_kegg_tumor')
normal_enr = gp.enrichr(gene_list=normal_genes, gene_sets='KEGG_2016', outdir='test/enrichr_kegg_normal')

We can view the data as a table by using the `obj.res2d` command (`obj` being the variable name specified in the previous step). The Gene Set Enrichment Analysis returns a list of pathways the genes provided are involved in, based on a significant adjusted P-value.

In [ ]:
tumor_enr.res2d.head()

In [ ]:
normal_enr.res2d.head()

## Step 6: Plot the p-values

We can better visualize these tables and the significant pathways detected through the gseapy imported `barplot()` function, which takes an enrichr table and a title as parameters.

In [ ]:
barplot(tumor_enr.res2d,title="Proteomics Tumor KEGG_2016")
plt.show()

In [ ]:
barplot(normal_enr.res2d,title="Normal KEGG_2016")
plt.show()

## Discussion

When comparing these barplots, we can observe the "Tumor" group has one pathway expressed at a very significant adjusted P-value, while the other pathways are expressed at a less significant P-value. The single pathway is the Ribosome pathway, which could be attributed to the "Tumor" genes disrupting ribosome activity. In comparison, the "Normal" group has a higher count of high significant P-values.